### **Imports**

In [ ]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import NearestNeighbors
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.feature_selection import chi2
from sko.PSO import PSO
from sklearn.feature_selection import chi2
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from scipy.stats import ttest_rel
from statistics import mean, stdev
import shap
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline
from collections import defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin


### **Data Ingestion**

In [ ]:
# Carregamento de Dados
test = pd.read_csv(r'../datasets/test_data_cleaned.csv')
train = pd.read_csv(r'../datasets/training_data_cleaned.csv')
train = train.drop(columns=['hour'])
test = test.drop(columns=['hour'])

### **Data Segregation**

In [ ]:
y = train['AVERAGE_SPEED_DIFF']
X = train.drop(columns=['AVERAGE_SPEED_DIFF'])
X_test = test

### **Model Training** 

#### *Specialists Models*

In [ ]:
classes = [0,1,2,3,4]
inverse_map = {0:'None', 1:'Low', 2:'Medium', 3:'High', 4:'Very_High'}

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2025)

# Store specialists per fold
specialists = {cls: [] for cls in classes}

# Store results per fold
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train each specialist in a One vs Rest way
    fold_preds = np.zeros((len(X_val_fold), len(classes)))
    
    for cls in classes:
        # Apply SMOTE
        sm = SMOTE(random_state=2025)
        # Alternatives
        #sm = SMOTEENN(random_state=2025)
        #sm = ADASYN(random_state=2025)

        # Split 
        X_train_res, y_train_res = sm.fit_resample(X_train_fold, (y_train_fold == cls).astype(int))

        # Define model
        model = RandomForestClassifier(n_estimators=118,max_depth=27,min_samples_split=2,min_samples_leaf=1,criterion='gini',class_weight="balanced",random_state=2025,n_jobs=-1)
        
        # Train and store model
        model.fit(X_train_res, y_train_res)
        specialists[cls].append(model)
        
        # Probability of a specialist in a validation fold
        fold_preds[:, cls] = model.predict_proba(X_val_fold)[:,1]
    
    # Final prediction (argmax of said probabilities)
    y_val_pred = np.argmax(fold_preds, axis=1)
    
    # Save metrics
    fold_results.append({"fold": fold,"accuracy": accuracy_score(y_val_fold, y_val_pred),"report": classification_report(y_val_fold, y_val_pred, output_dict=True)})
    
    # Show Folds
    print(classification_report(y_val_fold, y_val_pred))

# Accuracy and deviation
acc_list = [f['accuracy'] for f in fold_results]
print("\n=== Resumo Geral ===")
print(f"Accuracy média: {np.mean(acc_list):.4f}")
print(f"Desvio padrão: {np.std(acc_list):.4f}")

# F1-score per classe
metrics_per_class = defaultdict(list)

for f in fold_results:
    for cls in map(str, classes):
        metrics_per_class[cls].append(f['report'][cls]['f1-score'])

print("\nF1-score médio por classe:")
for cls in classes:
    print(f"{inverse_map[cls]}: {np.mean(metrics_per_class[str(cls)]):.4f} ± {np.std(metrics_per_class[str(cls)]):.4f}")


# Function to make predictions
def predict_specialists(X_input, specialists, classes):
    probs = []
    for cls in classes:
        prob_cls = np.mean([m.predict_proba(X_input)[:,1] for m in specialists[cls]], axis=0)
        probs.append(prob_cls)
    probs = np.vstack(probs).T
    y_pred = np.argmax(probs, axis=1)
    return y_pred, probs


### **Feature Importance**

In [ ]:
class SpecialistsEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self, specialists, classes):
        self.specialists = specialists
        self.classes = classes

    def fit(self, X, y=None):
        return self

    def predict_proba(self, X):
        probs = []
        for cls in self.classes:
            cls_probs = np.mean(
                [m.predict_proba(X)[:, 1] for m in self.specialists[cls]],
                axis=0
            )
            probs.append(cls_probs)

        probs = np.vstack(probs).T
        return probs

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

ensemble_model = SpecialistsEnsemble(
    specialists=specialists,
    classes=classes
)
from sklearn.inspection import permutation_importance
import time

start_time = time.time()

result = permutation_importance(ensemble_model,X_val_fold,y_val_fold,n_repeats=10,random_state=42,n_jobs=2)

elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
p_importances = pd.Series(
    result.importances_mean,
    index=X_val_fold.columns
).sort_values(ascending=False)

print("Feature importances using Permutation Importance:\n", p_importances)

fig, ax = plt.subplots(figsize=(16, 8))
p_importances.plot.barh(
    yerr=result.importances_std,
    ax=ax,
    color="grey"
)
ax.grid(axis="y")
ax.set_xlabel("Mean decrease in accuracy", fontsize=18)
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=14)
fig.tight_layout()
plt.show()


### **Submission Preparation**

In [ ]:
def generate_submission(input_csv_path,output_csv_path,specialists,classes,inverse_map,start_rowid=1):
    # Read CSV
    df = pd.read_csv(input_csv_path)
    X_input = df.copy()
    # Predictions
    y_pred, _ = predict_specialists(X_input, specialists, classes)

    # RowId based on line (to meet format)
    row_ids = np.arange(start_rowid, start_rowid + len(df))

    # Create submission
    submission = pd.DataFrame({
        "RowId": row_ids,
        "Speed_Diff": [inverse_map[c] for c in y_pred]
    })

    # Export
    submission.to_csv(output_csv_path, index=False)

    print(f"Submission criada: {output_csv_path}")

generate_submission(input_csv_path=r"../datasets/test_data_cleaned.csv",output_csv_path=r"../submission.csv",specialists=specialists,classes=classes,inverse_map=inverse_map)

### **Mixture of Experts (MoE)**

In [ ]:
classes = sorted(y.unique())
inverse_map = {0: "None",1: "Low",2: "Medium",3: "High",4: "Very_High"}

# Stratified
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,stratify=y,random_state=2025)

# Train loop of the MoE
def train_moe(X, y,n_experts=3,n_splits=5,random_state=2025):
    # Stratified k Fold
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)

    experts = [[] for _ in range(n_experts)]
    gates = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n=== Fold {fold} ===")

        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Oversampling (MULTICLASS)
        smote = SMOTE(random_state=random_state)
        X_res, y_res = smote.fit_resample(X_tr, y_tr)

        # Train experts
        for i in range(n_experts):
            expert = RandomForestClassifier(n_estimators=118,max_depth=27,min_samples_split=2,min_samples_leaf=1,criterion='gini',class_weight="balanced",random_state=random_state + i,n_jobs=-1)
            expert.fit(X_res, y_res)
            experts[i].append(expert)

        # Train gate
        gate = RandomForestClassifier(n_estimators=118,max_depth=27,min_samples_split=2,min_samples_leaf=1,criterion='gini',class_weight="balanced",random_state=random_state + i,n_jobs=-1)
        gate.fit(X_tr, y_tr)
        gates.append(gate)

    return experts, gates

# Predict with MoE
def predict_moe(X, experts, gates):
    n_experts = len(experts)
    n_samples = len(X)
    n_classes = experts[0][0].n_classes_

    # Expert probabilities
    expert_probs = np.zeros((n_experts, n_samples, n_classes))

    for i, expert_list in enumerate(experts):
        fold_probs = [m.predict_proba(X) for m in expert_list]
        expert_probs[i] = np.mean(fold_probs, axis=0)

    # Gate weights
    gate_weights = np.mean([g.predict_proba(X) for g in gates],axis=0)  # shape: (n_samples, n_classes)

    # Normalize to expert count
    gate_weights = gate_weights[:, :n_experts]
    gate_weights = gate_weights / gate_weights.sum(axis=1, keepdims=True)

    # Mixture
    final_probs = np.zeros((n_samples, n_classes))
    for i in range(n_experts):
        final_probs += gate_weights[:, i, None] * expert_probs[i]

    y_pred = np.argmax(final_probs, axis=1)
    return y_pred, final_probs


# Call training
experts, gates = train_moe(X_train,y_train,n_experts=5,n_splits=5)

# Call predict
y_test_pred, y_test_probs = predict_moe(X_test, experts, gates)

# Metrics
print("\n=== FINAL MODEL PERFORMANCE ===\n")
print(classification_report(y_test,y_test_pred,target_names=[inverse_map[c] for c in classes],digits=4))

accuracy = accuracy_score(y_test, y_test_pred)
print(f"Overall Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[inverse_map[c] for c in classes])
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix – Mixture of Experts")
plt.tight_layout()
plt.show()

### **Importance**

In [ ]:
class MoEEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self, experts, gates):
        self.experts = experts
        self.gates = gates

    def fit(self, X, y=None):
        return self

    def predict_proba(self, X):
        n_experts = len(self.experts)
        n_samples = len(X)
        n_classes = self.experts[0][0].n_classes_

        # Expert probabilities
        expert_probs = np.zeros((n_experts, n_samples, n_classes))
        for i, expert_list in enumerate(self.experts):
            fold_probs = [m.predict_proba(X) for m in expert_list]
            expert_probs[i] = np.mean(fold_probs, axis=0)

        # Gate weights
        gate_weights = np.mean(
            [g.predict_proba(X) for g in self.gates],
            axis=0
        )

        # Normalize to number of experts
        gate_weights = gate_weights[:, :n_experts]
        gate_weights = gate_weights / gate_weights.sum(axis=1, keepdims=True)

        # Mixture
        final_probs = np.zeros((n_samples, n_classes))
        for i in range(n_experts):
            final_probs += gate_weights[:, i, None] * expert_probs[i]

        return final_probs

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
moe_model = MoEEnsemble(experts=experts,gates=gates)

start_time = time.time()

result = permutation_importance(moe_model,X_test,y_test,n_repeats=10,random_state=42,n_jobs=2)

elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

p_importances = (
    pd.Series(result.importances_mean, index=X_test.columns)
      .sort_values(ascending=False)
)

print("Feature importances using Permutation Importance:\n", p_importances)

fig, ax = plt.subplots(figsize=(16, 8))
p_importances.plot.barh(yerr=result.importances_std,ax=ax,color="grey")
ax.grid(axis="y")
ax.set_xlabel("Mean decrease in accuracy", fontsize=18)
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=14)
fig.tight_layout()
plt.show()

### **Submission Preparation**

In [ ]:
def generate_submission(input_csv_path,output_csv_path,experts,gates,inverse_map,start_rowid=1):
    # Read CSV
    df = pd.read_csv(input_csv_path)
    X_input = df.copy()

    # Predict with MoE
    y_pred, _ = predict_moe(X_input, experts, gates)

    # RowId based on line number
    row_ids = np.arange(start_rowid, start_rowid + len(df))

    # Create submission
    submission = pd.DataFrame({
        "RowId": row_ids,
        "Speed_Diff": [inverse_map[int(c)] for c in y_pred]
    })

    # Export
    submission.to_csv(output_csv_path, index=False)

    print(f"Submission criada: {output_csv_path}")

generate_submission(input_csv_path=r"../datasets/test_data_cleaned.csv",output_csv_path=r"../submission.csv",experts=experts,gates=gates,inverse_map=inverse_map)